<a href="https://colab.research.google.com/github/Bombbom/ML4SecurityProject/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM 



In [ ]:
import pandas as pd
data = pd.read_pickle("/content/drive/MyDrive/DACN/dataset_fragment_vectors.pkl")

In [ ]:
from __future__ import print_function

import warnings
import numpy as np

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, LSTM, ReLU, Activation, Bidirectional
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adamax
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight, compute_class_weight

#from parser import parameter_parser
from tensorflow.keras.layers import Layer
#from models.loss_draw import LossHistory
from keras import backend as K
from keras import initializers, regularizers, constraints

In [ ]:
vectors = np.stack(data.iloc[:, 0].values)
# vectors = vectors.reshape()
labels = data.iloc[:, 1].values

In [ ]:
positive_idxs = np.where(labels == 1)[0]
negative_idxs = np.where(labels == 0)[0]
idxs = np.concatenate([positive_idxs, negative_idxs])
undersampled_negative_idxs = np.random.choice(negative_idxs, len(positive_idxs), replace=True)
resampled_idxs = np.concatenate([positive_idxs, undersampled_negative_idxs])
x_train, x_test, y_train, y_test = train_test_split(vectors[resampled_idxs], labels[resampled_idxs],train_size=0.8,random_state=1, stratify=labels[resampled_idxs])

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
dropout = 0.2
# y_train = to_categorical(y_train)
lr = 0.002
batch_size = 64
epochs = 10
threshold = 0.5
  
class_weights =  class_weight.compute_class_weight(class_weight='balanced', classes=[0, 1], y=labels)
adamax = Adamax(lr)
model = Sequential()
model.add(LSTM(300, input_shape=(vectors.shape[1], vectors.shape[2])))
model.add(ReLU())
model.add(Dropout(dropout))
model.add(Dense(300))
model.add(ReLU())
model.add(Dropout(dropout))
model.add(Dense(2, activation='softmax'))

In [ ]:
#Tính và lưu trữ kết quả accuracy
model.compile(loss='binary_crossentropy', optimizer=adamax, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

In [ ]:
_, accuracy_test = model.evaluate(x_test, y_test)
_, accuracy_train = model.evaluate(x_train, y_train)
values = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
predictions = (model.predict(x_test, batch_size=batch_size)).round()
tn, fp, fn, tp = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1)).ravel()
FPR=fp / (fp + tn)
FNR= fn / (fn + tp)
recall = tp / (tp + fn)
precision = tp / (tp + fp)
F1=(2 * precision * recall) / (precision + recall)
storeResults("LSTM",(accuracy_train*100),(accuracy_test*100),recall*100,FPR*100,precision*100, F1*100)